# My not very good Neural Network

This is a little neural network program. I don't think it's very good. It's slow too.

Also, the code is way, way longer than I think it ought to be. However, most of the code is initialization.

In [279]:
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split

define activation functions

In [11]:
logistic=lambda x:1/(1+np.exp(-x))

d_logistic= lambda result: result*(1-result)

### Network classes

The main work here is done by the HiddenLayer class, which computes an output based on the previous layers input, and does the backpropagation.

In [346]:
class HiddenLayer(object):
    def __init__(self,n_in, n_out, weights=None,bias=None):
        #Is there a better way to make initilization less long
        if weights is None:
            weights=(2*np.random.random((n_in,n_out))-1)
        if bias is None:
            bias=(np.zeros((n_out),dtype=np.float64))
        self.n_in=n_in
        self.n_out=n_out
        self.weights=weights
        self.bias=bias
        self.output=np.zeros((n_out),dtype=np.float64)
        self.inputs=np.zeros((n_in),dtype=np.float64)
        self.delta=np.zeros((n_in),dtype=np.float64)
        self.error=np.zeros((n_out),dtype=np.float64)
    def forward_prop(self,inputs):
        #This is the important part
        ins=self.bias+np.dot(inputs,self.weights)
        #ins=np.dot(inputs,self.weights)
        self.output = logistic(ins)
        self.inputs=inputs
        #return self.output
    def backprop(self,delta,alpha):
        #this is also important
        self.error=np.array(delta.dot(np.matrix(self.bias).T+self.weights.T))
        #self.error=np.array(delta.dot(self.weights.T))
        self.back_delta=self.error*d_logistic(self.inputs)
        self.weights+=alpha*self.inputs.T.dot(delta)
        self.bias-=alpha*delta.sum(axis=0)
        #return self.back_delta
        
        

I'm not 100% satisfied with the logic in making the output a whole separate layer. All it does is take in the input from the previous layer and compute the error. However, this is how an people draw a Neural Network.

In [365]:
class OutputLayer(object):
    def __init__(self,y):
        if (len(y.shape)>1):
            n_in=y.shape[-1]
        else:
            #needed in case you want to input a pandas series
            y=np.expand_dims(y,axis=1)
            n_in=1
        self.n_in=n_in
        self.y=y
        self.inputs=np.zeros((n_in),dtype=np.float64)
    def forward_prop(self,inputs):
        self.inputs=inputs
    def backprop(self,alpha):
        self.error=self.y-self.inputs
        self.back_delta=self.error*d_logistic(self.inputs)
        

This creates the Neural Network itself. The network (`Multi_Layer_Perceptron().layers`) is a list of `Layer` classes that I defined above.

In [348]:
class Multi_Layer_Perceptron(object):
    def __init__(self,n_layers,features,y,alpha=0.01, n_hidden_paths=None):
        self.alpha=alpha
        self.features=features
        self.n_layers=n_layers
        #Set the number of features
        if (len(features.shape)>1):
            self.n_features=features.shape[-1]
        else:
            self.n_features=1
        #Set the number of output features
        if (len(y.shape)>1):
            n_out=y.shape[-1]
        else:
            n_out=1
        if n_hidden_paths==None:
            n_hidden_paths=self.n_features+1
        #Initialize the layers of the network
        if n_layers>2:
            self.layers=[HiddenLayer(self.n_features,n_hidden_paths)]
            if n_layers>3:
                self.layers.extend([HiddenLayer(n_hidden_paths,n_hidden_paths) for i in xrange(n_layers-3)])
            self.layers.extend([HiddenLayer(n_hidden_paths,n_out),OutputLayer(y)])
        elif n_layers==2:
            self.layers=[HiddenLayer(self.n_features,n_out),OutputLayer(y)]
        else:
            raise ValueError('Network must have at least 2 layers, an input and output layer.')
    def forward_prop(self,features=None):
        if features is None:
            features=self.features
        self.layers[0].forward_prop(features)
        for i in xrange(1,self.n_layers):
            self.layers[i].forward_prop(self.layers[i-1].output)
    def backprop(self,alpha=None):
        if alpha is not None:
            self.alpha=alpha
        self.layers[-1].backprop(self.alpha)
        for i in reversed(xrange(0,self.n_layers-1)):
            self.layers[i].backprop(self.layers[i+1].back_delta,self.alpha)
    def error(self):
        return (np.abs(np.round(self.layers[-1].error))).mean()
            
        

The main purpose of `MLP_Classifier` is to have a syntax that will play well with `sklearn`, so I can use it with `GridSearchCV`, for example. I haven't actually figured out how to get it to play nice with any of those though. It would be great to figure out how to do that.

It does the work of running the iterations of forward and backward propogation, in the `fit` method.

In [359]:
class MLP_Classifier(object):
    def __init__(self,n_layers=4,alpha=0.01, n_hidden_paths=None,min_it=100,max_it=10000,min_error_delta=1e-10,iter_check=50):
        self.n_layers=n_layers
        self.alpha=alpha
        self.n_hidden_paths=n_hidden_paths
        self.min_it=min_it
        self.max_it=max_it
        self.min_error_delta=min_error_delta
        self.iter_check=iter_check
        self.iterations=0
    def fit(self,features,y):
        self.error_delta=1.0
        self.last_error=1.0
        self.MLPC=Multi_Layer_Perceptron(self.n_layers,features,y,alpha=self.alpha,n_hidden_paths=self.n_hidden_paths)
        while (self.iterations<self.max_it) and (((self.error_delta)>self.min_error_delta) or (self.error_delta<0.0)):
            for i in xrange(self.iter_check):
                self.MLPC.forward_prop()
                self.MLPC.backprop()
                self.iterations+=1
            self.error=self.MLPC.error()
            if self.iterations>self.min_it:
                self.error_delta=self.last_error-self.error
            self.last_error=self.error             
    def score(self):
        #This doesn't actually compute a useful prediction score. Ooops.
        if self.iterations<=0:
            return 'You need to fit first'
        else:
            return self.MLPC.error()
    def predict(self,feat):
        if self.iterations<=0:
            return 'You need to fit first'
        else:
            self.MLPC.forward_prop(features=feat)
            return (self.MLPC.layers[-1].inputs)
        
    

### Some test data

First I'm going to try this out on a super easy data set: Just some of the 0's and 1's in our digit data set.

My computer can't quite have enough memory to handle the whole data set.

In [291]:
with open('../datapath.txt') as f:
    datapath=f.readlines()[0].rstrip()

In [314]:
train=pd.read_csv(datapath+'/train.csv', nrows=5000, dtype=np.uint8)
pixels=train.columns.drop('label')

In [315]:
train=train[(train['label']==1)|(train['label']==0)]

In [316]:
train.shape

(1052, 785)

I'm just going to run it for 1000 iterations or until it's not getting any better.

In [360]:
MLP=MLP_Classifier(max_it=1000,alpha=0.001,min_error_delta=0.0)

In [318]:
train,test=train_test_split(train,test_size=0.25)

In [361]:
%time MLP.fit(train[pixels],train['label'])

CPU times: user 3min 16s, sys: 412 ms, total: 3min 17s
Wall time: 3min 17s


In [362]:
MLP.iterations

200

In [363]:
errors=np.round(np.abs(MLP.predict(test[pixels]).flatten()-test['label']))
errors.mean()

0.019011406844106463

I'm not going to lie: 2% wrong is pretty sucky. I bet I could do a lot better, and random forest probably can too.